In [14]:
import os
import numpy as np
import cv2

from matplotlib import pyplot as plt
import matplotlib 

from time import time
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

from skimage.feature import hog

### Cargado de datos

Organizamos las imágenes en carpetas para poder procesarlas correctamente

In [15]:
import pandas as pd
from shutil import copyfile

csv_path = "C:/Users/lllrm/Downloads/dataset_VC/fairface_label_train.csv"
output_dir = "C:/Users/lllrm/Downloads/dataset_VC/ages"
image_path = "C:/Users/lllrm/Downloads/dataset_VC"

df = pd.read_csv(csv_path)

for _, row in df.iterrows():
    age_group = row['age']
    src = os.path.join(image_path, row['file'])  # original image path
    dst = os.path.join(output_dir, age_group, os.path.basename(row['file']))  # ignore train/test subfolder

    # Make sure the destination folder exists
    os.makedirs(os.path.dirname(dst), exist_ok=True)

    # Copy the file if it exists
    if os.path.exists(src):
        copyfile(src, dst)

In [15]:
folder = "C:/Users/lllrm/Downloads/dataset_VC/train"

nclasses = 0
nperclass = []
classlabels = []
X = []
Y = []

dpi = matplotlib.rcParams['figure.dpi']

for class_name in os.listdir(folder):
    nclasses += 1
    nsamples = 0

    class_folder = os.path.join(folder, class_name)
    for file_name in os.listdir(class_folder):
        if file_name.endswith('.jpg'):
            image = cv2.imread (os.path.join(class_folder, file_name),)
            image = cv2.resize(image, (128, 128))
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            features = hog(gray, 
                           orientations=9,
                           pixels_per_cell=(8, 8),
                           cells_per_block=(2, 2),
                           block_norm='L2-Hys')

            X.append(features)
            Y.append(nclasses-1)
  
            nsamples += 1
    nperclass.append(nsamples)
    classlabels.append(class_name)
X = np.array(X,dtype='float32')
Y = np.array(Y,dtype='float64')


n_samples , n_features = X.shape

class_names = np.array(classlabels)
n_classes = class_names.shape[0]

print("Dataset info:")
print("# features: %d" % n_features)
print("# classes: %d" % n_classes)
print("classes %s" % classlabels)
print("samples per class %s" % str(nperclass)[1:-1] )

Dataset info:
# features: 8100
# classes: 5
classes ['18-20', '21-30', '31-40', '41-50', '51-60']
samples per class 30, 30, 30, 30, 30


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42)

print("# samples in training set %d" % X_train.shape[0])
print("# samples in test set %d" % X_test.shape[0])

# samples in training set 105
# samples in test set 45


In [21]:
print("SVM classifier based on HOG\nTraining...")
t0 = time()

model_svm = SVC(kernel='rbf', C=5, gamma='scale')
model_svm.fit(X_train, y_train)

print("done in %0.3fs" % (time() - t0))

print("Predicting...")
t0 = time()
y_pred = model_svm.predict(X_test)
print("done in %0.3fs" % (time() - t0))

print("Classification results:")
print(classification_report(y_test, y_pred, target_names=class_names))

print("Precision:  %0.3f, Recall:  %0.3f" %
      (precision_score(y_test, y_pred, average='macro'),
       recall_score(y_test, y_pred, average='macro')))

print("Confusion matrix")
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))


SVM classifier based on HOG
Training...
done in 0.079s
Predicting...
done in 0.037s
Classification results:
              precision    recall  f1-score   support

       18-20       0.67      0.15      0.25        13
       21-30       0.50      0.33      0.40         9
       31-40       0.25      0.20      0.22        10
       41-50       0.14      0.75      0.24         4
       51-60       0.14      0.11      0.12         9

    accuracy                           0.24        45
   macro avg       0.34      0.31      0.25        45
weighted avg       0.39      0.24      0.25        45

Precision:  0.340, Recall:  0.310
Confusion matrix
[[2 3 3 3 2]
 [1 3 1 3 1]
 [0 0 2 6 2]
 [0 0 0 3 1]
 [0 0 2 6 1]]


## Entrenamiento

### Tarea 2

In [1]:
import cv2
from deepface import DeepFace
import json
import random

In [ ]:
def text(content, frame, y, color):
    font = cv2.FONT_HERSHEY_SIMPLEX
    thickness = 1
    max_width = frame.shape[1] - 40  
    scale = 0.7
 
    (left_width, left_height), left_baseline = cv2.getTextSize(left_text, font, scale, thickness)
    (right_width, right_height), right_baseline = cv2.getTextSize(right_text, font, scale, thickness)
 
 
    (text_width, text_height), baseline = cv2.getTextSize(content, font, scale, thickness)
    while text_width > max_width and scale > 0.1:
        scale -= 0.01
        (text_width, text_height), _ = cv2.getTextSize(content, font, scale, thickness)
 
    frame_width = frame.shape[1]
    x_text = (frame_width - text_width) // 2
    y_text = y
    cv2.putText(frame, content, (x_text, y_text), font, scale, color, thickness)
 
# Webcam connection
cap = cv2.VideoCapture(0)
 
x_offset = 0
is_selected = None
new_question = False
answer = None
delay = 0
index_pregunta = 0
score = 0

preguntas = list()
with open("preguntas.json", "r", encoding="utf-8") as f:
    file = json.load(f)
    
preguntas = file["preguntas"]
pregunta = None
while True:
    # Read frame
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)  # Flip frame horizontally for mirror effect
    if not ret:
        break
 
    try:
        # Face detection
        faces = DeepFace.extract_faces(frame, detector_backend='opencv')
        #print(faces)

        if index_pregunta >= len(preguntas):
            text("Se han terminado las preguntas.", frame, 140, (0,0,0))
 
        # Draw line in the middle
        cv2.line(frame, (frame.shape[1]//2, 0), (frame.shape[1]//2, frame.shape[0]), (255, 0, 0), 1)
        if pregunta is None or new_question:
            pregunta = preguntas[index_pregunta]
            index_pregunta += 1
            new_question = False
       
        # Show question text
        font = cv2.FONT_HERSHEY_SIMPLEX
        thickness = 1
        max_width = frame.shape[1] - 40  
        scale = 0.7
 
        left_text = "verdadero"
        right_text = "falso"

        cv2.rectangle(frame, (0, 0), (frame.shape[1], text_height + baseline + 50), (255, 255, 255), -1)
 
        text(pregunta["pregunta"], frame, 50, (0,0,0))
        
        # Show True text
        padding = 10
        x_left = 10
        (left_width, left_height), left_baseline = cv2.getTextSize(left_text, font, scale, thickness)
        top_left = frame.shape[0] - 20 - left_height - padding // 2
        bottom_left = frame.shape[0] - 20 + left_baseline + padding // 2
        cv2.rectangle(frame, (x_left - padding, top_left), (x_left + left_width + padding, bottom_left), (0, 0, 0), -1)
        cv2.putText(frame, left_text, (x_left, frame.shape[0] - 20), font, scale, (0, 255, 0), thickness)
 
        # Show False text
        x_right = frame.shape[1] - right_width - 10
        (right_width, right_height), right_baseline = cv2.getTextSize(right_text, font, scale, thickness)
        top_right = frame.shape[0] - 20 - right_height - padding // 2
        bottom_right = frame.shape[0] - 20 + right_baseline + padding // 2
        cv2.rectangle(frame, (x_right - padding, top_right), (x_right + right_width + padding, bottom_right), (0, 0, 0), -1)
        cv2.putText(frame, right_text, (x_right, frame.shape[0] - 20), font, scale, (0, 0, 255), thickness)

        # Print score
        score_text = f"Score: {score}"
        cv2.putText(frame, score_text, (x_left, 100), font, scale, (0, 0, 0), 2)

 

        # Show correct/incorrect answer texti
        if answer == 0:
            text("Respuesta correcta", frame, 100, (0, 255, 0))
            delay += 1
        elif answer == 1:
            text("Respuesta incorrecta", frame, 100, (0, 0, 255))
            delay += 1
        elif answer == 2:
            text("Respuesta correcta", frame, 100, (0, 255, 0))
            delay += 1
        elif answer == 3:
            text("Respuesta incorrecta", frame, 100, (0, 0, 255))
            delay += 1
        
        if delay >= 25:
            answer = None
            delay = 0
 
        frame_width = frame.shape[1]
 
        # Draw face bounding box and eyes locations
        for face in faces:
            x, y, w, h = face['facial_area']['x'], face['facial_area']['y'], face['facial_area']['w'], face['facial_area']['h']
            # Save starting position for the first frame
            if x0 == 0:
                x0 = frame_width // 2
            # Izquierda
            elif x_offset < -(0.1*frame_width):
                # Consider that the person has tilted their head towards the left
                x_offset = 0
                # Dibujar un rectángulo sin relleno
                cv2.rectangle(frame, (x_left - padding, top_left), (x_left + left_width + padding, bottom_left), (255, 255, 255), 2)
                is_selected = 0
            # Derecha
            elif x_offset > (0.1*frame_width):
                # Consider that the person has tilted their head towards the right
                x_offset = 0   
                cv2.rectangle(frame, (x_right - padding, top_right), (x_right + right_width + padding, bottom_right), (255, 255, 255), 2) 
                is_selected = 1
            else:
                x_offset = 0
                if is_selected is not None:
                    # Mostramos la respuesta correcta
                    if is_selected == 0 and pregunta["respuesta_correcta"] == "Verdadero":
                        answer = 0
                        score += 1
                    elif is_selected == 0 and pregunta["respuesta_correcta"] == "Falso":
                        answer = 1
                    elif is_selected == 1 and pregunta["respuesta_correcta"] == "Falso":
                        answer = 2
                        score += 1
                    elif is_selected == 1 and pregunta["respuesta_correcta"] == "Verdadero":
                        answer = 3
                    is_selected = None
                    new_question = True                    
            x_offset += (x + w//2) - x0
            cv2.rectangle(frame, (x, y), (x + w, y + h), (200, 255, 200), 2)
 
    except Exception as e:
        print("Error:", e)
    cv2.imshow("Video", frame)
 
    # Salir si se presiona Esc o cambia el detectr
    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # Salir si se presiona 'q'
        break
 
# Liberar la captura y cerrar ventanas
cap.release()
cv2.destroyAllWindows()

Respuesta correcta: Verdadero
Respuesta correcta: Falso
